## **Quantile**

## **Mengimpor libaries** 
Dalam bagian ini, kita akan mengimpor beberapa library yang diperlukan untuk memproses data dalam format JSON serta untuk analisis dan visualisasi data. Library yang akan kita gunakan adalah:

- **pandas**: Untuk manipulasi dan analisis data.
- **numpy**: Untuk operasi numerik dan manipulasi array.
- **json**: Untuk bekerja dengan data dalam format JSON.
- **matplotlib.pyplot**: Untuk visualisasi data.
- **seaborn**: Untuk visualisasi data yang lebih menarik dan informatif.

In [1]:
# Mengimpor libaries pandas,numpy dan json untuk memproses data dalam format JSON
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns

# import warning to ignore warning
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

## **Install mysql connector**

In [2]:
# install mysql connector
!pip install mysql-connector-python

In [3]:
# import libraries
import mysql.connector
from mysql.connector import Error

In [4]:
# Define MySQL details
host   = 'okegarden-laravel-prod-db-2024-11-15-0200.cgwhhgh85mr3.ap-southeast-3.rds.amazonaws.com'
port   = '3306'
user   = 'intern_mes'
passwd = 'sdfDFLES342982Ddlsdksd'
databs = 'okegarden_laravel_prod_db'

In [5]:
# function to fetch data from database, the connection is closed after use.
def fetch_data(host_name, port_name, user_name, user_password, db_name, query, params=None):
    try:
        # use with to ensure the connection is closed after use
        with mysql.connector.connect(
            host=host_name,
            port=port_name,
            user=user_name,
            password=user_password,
            database=db_name
        ) as connection:
            data = pd.read_sql(query, connection, params=params) # read the query to dataframe format
            return data

    except Error as e:
        print(f"Error while connecting to MySQL or fetching data: {e}")
        return None

## **Query Data**
#### Data ini merupakan hasil dari proses assessment yang telah kami lakukan sebelumnya.

In [6]:
query = """

WITH order_garden_data AS (
    SELECT
        act.id,
        act.subject_type,
        act.subject_id,
        act.properties,
        act.created_at,
        act.updated_at,
        JSON_UNQUOTE(JSON_EXTRACT(act.properties, '$.old.status')) AS old_status,
        JSON_UNQUOTE(JSON_EXTRACT(properties, '$.old.created_at')) AS old_created_at,
        JSON_UNQUOTE(JSON_EXTRACT(properties, '$.old.updated_at')) AS old_updated_at,
        JSON_UNQUOTE(JSON_EXTRACT(act.properties, '$.attributes.status')) AS attributes_status,
        JSON_UNQUOTE(JSON_EXTRACT(properties, '$.attributes.created_at')) AS attributes_created_at,
        JSON_UNQUOTE(JSON_EXTRACT(properties, '$.attributes.updated_at')) AS attributes_updated_at,
        TIMESTAMPDIFF(SECOND, 
            JSON_UNQUOTE(JSON_EXTRACT(properties, '$.attributes.created_at')),
            JSON_UNQUOTE(JSON_EXTRACT(properties, '$.attributes.updated_at')) 
        ) / 3600 AS order_cycle_time,
        dt.id AS id_datetime,
        TIMESTAMPDIFF(SECOND,
            JSON_UNQUOTE(JSON_EXTRACT(properties, '$.old.updated_at')),
            act.created_at
        ) / 3600 AS time_interval_status
    FROM
        activity_log act
    LEFT JOIN
        datetime dt ON act.subject_id = dt.datetimeable_id
    WHERE 
        subject_type LIKE '%OrderDesign%' 
        OR subject_type LIKE '%OrderGarden%'
        OR subject_type LIKE '%OrderGardenCareDetail%'
        
),
build_start_attributes_status AS (
    SELECT
        act.id,
        act.subject_type,
        act.subject_id,
        act.properties,
        act.created_at,
        dt.updated_at,
        CASE
            WHEN JSON_UNQUOTE(JSON_EXTRACT(act.properties, '$.old.status')) = 'acc_gardener' 
                 AND JSON_UNQUOTE(JSON_EXTRACT(act.properties, '$.attributes.status')) = 'in_review'
            THEN 'acc_gardener'
        END AS old_status,
        JSON_UNQUOTE(JSON_EXTRACT(act.properties, '$.old.created_at')) AS old_created_at,
        JSON_UNQUOTE(JSON_EXTRACT(act.properties, '$.old.updated_at')) AS old_updated_at,
        CASE
            WHEN JSON_UNQUOTE(JSON_EXTRACT(act.properties, '$.old.status')) = 'acc_gardener' 
                 AND JSON_UNQUOTE(JSON_EXTRACT(act.properties, '$.attributes.status')) = 'in_review'
            THEN 'build_start'
        END AS attributes_status,
        dt.created_at AS attributes_created_at,
        dt.updated_at AS attributes_updated_at,
        TIMESTAMPDIFF(
            SECOND, 
            dt.created_at,
            dt.updated_at
        ) / 3600 AS order_cycle_time,
        dt.id AS id_datetime,
        TIMESTAMPDIFF(
            SECOND,
            JSON_UNQUOTE(JSON_EXTRACT(act.properties, '$.old.updated_at')),
            act.created_at
        ) / 3600 AS time_interval_status
    FROM
        activity_log act
    LEFT JOIN
        datetime dt ON act.subject_id = dt.datetimeable_id
    WHERE 
        (
            subject_type LIKE '%OrderDesign%' 
            OR subject_type LIKE '%OrderGarden%'
            OR subject_type LIKE '%OrderGardenCareDetail%'
        )
        AND JSON_UNQUOTE(JSON_EXTRACT(act.properties, '$.old.status')) = 'acc_gardener'
        AND JSON_UNQUOTE(JSON_EXTRACT(act.properties, '$.attributes.status')) = 'in_review'
),
build_start_old_status AS (
    SELECT
        act.id,
        act.subject_type,
        act.subject_id,
        act.properties,
        dt.created_at,
        dt.updated_at,
        CASE
            WHEN JSON_UNQUOTE(JSON_EXTRACT(act.properties, '$.old.status')) = 'acc_gardener' 
                 AND JSON_UNQUOTE(JSON_EXTRACT(act.properties, '$.attributes.status')) = 'in_review'
            THEN 'build_start' 
        END AS old_status,
        dt.created_at AS old_created_at,
        dt.datetime AS old_updated_at,
        CASE
            WHEN JSON_UNQUOTE(JSON_EXTRACT(act.properties, '$.old.status')) = 'acc_gardener' 
                 AND JSON_UNQUOTE(JSON_EXTRACT(act.properties, '$.attributes.status')) = 'in_review'
            THEN 'in_review'
        END AS attributes_status,
        JSON_UNQUOTE(JSON_EXTRACT(act.properties, '$.attributes.created_at')) AS attributes_created_at,
        JSON_UNQUOTE(JSON_EXTRACT(act.properties, '$.attributes.updated_at')) AS attributes_updated_at,
        TIMESTAMPDIFF(
            SECOND, 
            JSON_UNQUOTE(JSON_EXTRACT(act.properties, '$.attributes.created_at')),
            JSON_UNQUOTE(JSON_EXTRACT(act.properties, '$.attributes.updated_at'))
        ) / 3600 AS order_cycle_time,
        dt.id AS id_datetime,
        TIMESTAMPDIFF(
            SECOND,
            dt.created_at,
            dt.datetime
        ) / 3600 AS time_interval_status
    FROM
        activity_log act
    LEFT JOIN
        datetime dt ON act.subject_id = dt.datetimeable_id
    WHERE 
    (
        subject_type LIKE '%OrderDesign%' 
        OR subject_type LIKE '%OrderGarden%'
        OR subject_type LIKE '%OrderGardenCareDetail%'
    )
    AND JSON_UNQUOTE(JSON_EXTRACT(act.properties, '$.old.status')) = 'acc_gardener'
    AND JSON_UNQUOTE(JSON_EXTRACT(act.properties, '$.attributes.status')) = 'in_review'
)
SELECT * FROM (
SELECT * FROM order_garden_data
UNION ALL
SELECT * FROM build_start_attributes_status
UNION ALL
SELECT * FROM build_start_old_status) AS combined_data
ORDER BY id DESC;

"""
order_process = fetch_data(host, port, user, passwd, databs, query)

In [7]:
order_process.head()

,id,subject_type,subject_id,properties,created_at,updated_at,old_status,old_created_at,old_updated_at,attributes_status,attributes_created_at,attributes_updated_at,order_cycle_time,id_datetime,time_interval_status
0,16519,App\Models\OrderDesign,1018,"{""attributes"":{""id_order"":1018,""id_user"":1968,...",2024-11-15 08:47:27,2024-11-15 08:47:27,None,None,None,paid,2024-11-15T01:47:27.000000Z,2024-11-15T01:47:27.000000Z,0.0000,NaN,NaN
1,16515,App\Models\OrderDesign,1017,"{""attributes"":{""id_order"":1017,""id_user"":1967,...",2024-11-15 08:32:14,2024-11-15 08:32:14,None,None,None,paid,2024-11-15T01:32:14.000000Z,2024-11-15T01:32:14.000000Z,0.0000,NaN,NaN
2,16509,App\Models\OrderGardenCareDetail,1160,"{""attributes"":{""id"":1160,""id_order_garden_care...",2024-11-14 22:36:51,2024-11-14 22:36:51,paid,2024-10-06T10:18:37.000000Z,2024-11-02T03:55:18.000000Z,acc_gardener,2024-10-06T10:18:37.000000Z,2024-11-14T15:36:51.000000Z,941.3039,NaN,306.6925
3,16478,App\Models\OrderGardenCareDetail,976,"{""attributes"":{""id"":976,""id_order_garden_care""...",2024-11-14 22:24:25,2024-11-14 22:24:25,paid,2024-09-21T05:12:37.000000Z,2024-10-30T06:31:36.000000Z,acc_gardener,2024-09-21T05:12:37.000000Z,2024-11-14T15:24:25.000000Z,1306.1967,NaN,375.8803
4,16475,App\Models\OrderGardenCare,291,"{""attributes"":{""id"":291,""id_user"":1207,""id_gar...",2024-11-14 21:00:23,2024-11-14 21:00:23,on_going,2024-11-13T03:48:28.000000Z,2024-11-14T13:36:27.000000Z,in_review,2024-11-13T03:48:28.000000Z,2024-11-14T14:00:23.000000Z,34.1986,NaN,7.3989


In [8]:
# Memfilter data berdasarkan subject_type
proses_subject_type = order_process[order_process['subject_type'].isin(['App\Models\OrderDesign','App\Models\OrderGarden','App\Models\OrderGardenCareDetail'])]

Dalam proyek ini, kami menggunakan hasil join tabel **activity_log** dan tabel **datetime** untuk mengambil semua perubahan data order yang terjadi pada tiga tabel utama, yaitu:

1. **OrderDesign**
2. **OrderGarden**
3. **OrderGardenCareDetail**

In [9]:
# Mengambil nilai unik dari kolom `subject_type`
proses_subject_type['subject_type'].unique()

array(['App\\Models\\OrderDesign', 'App\\Models\\OrderGardenCareDetail',
       'App\\Models\\OrderGarden'], dtype=object)

In [10]:
# Filter baris pada dataframe proses_subject_type yang memiliki perubahan status
# (yaitu atribut 'attributes_status' tidak sama dengan 'old_status')
proses_subject_type = proses_subject_type[proses_subject_type['attributes_status'] != proses_subject_type['old_status']]

In [11]:
# Hapus baris duplikat pada dataframe proses_subject_type berdasarkan kolom 'subject_id', 'old_status', dan 'attributes_status'
# Jaga hanya baris pertama dari setiap grup duplikat
proses_subject_type = proses_subject_type.drop_duplicates(subset=['subject_id','old_status','attributes_status'], keep='first')

In [12]:
proses_subject_type.head()

,id,subject_type,subject_id,properties,created_at,updated_at,old_status,old_created_at,old_updated_at,attributes_status,attributes_created_at,attributes_updated_at,order_cycle_time,id_datetime,time_interval_status
0,16519,App\Models\OrderDesign,1018,"{""attributes"":{""id_order"":1018,""id_user"":1968,...",2024-11-15 08:47:27,2024-11-15 08:47:27,None,None,None,paid,2024-11-15T01:47:27.000000Z,2024-11-15T01:47:27.000000Z,0.0000,NaN,NaN
1,16515,App\Models\OrderDesign,1017,"{""attributes"":{""id_order"":1017,""id_user"":1967,...",2024-11-15 08:32:14,2024-11-15 08:32:14,None,None,None,paid,2024-11-15T01:32:14.000000Z,2024-11-15T01:32:14.000000Z,0.0000,NaN,NaN
2,16509,App\Models\OrderGardenCareDetail,1160,"{""attributes"":{""id"":1160,""id_order_garden_care...",2024-11-14 22:36:51,2024-11-14 22:36:51,paid,2024-10-06T10:18:37.000000Z,2024-11-02T03:55:18.000000Z,acc_gardener,2024-10-06T10:18:37.000000Z,2024-11-14T15:36:51.000000Z,941.3039,NaN,306.6925
3,16478,App\Models\OrderGardenCareDetail,976,"{""attributes"":{""id"":976,""id_order_garden_care""...",2024-11-14 22:24:25,2024-11-14 22:24:25,paid,2024-09-21T05:12:37.000000Z,2024-10-30T06:31:36.000000Z,acc_gardener,2024-09-21T05:12:37.000000Z,2024-11-14T15:24:25.000000Z,1306.1967,NaN,375.8803
5,16474,App\Models\OrderGardenCareDetail,1480,"{""attributes"":{""id"":1480,""id_order_garden_care...",2024-11-14 21:00:23,2024-11-14 21:00:23,acc_gardener,2024-11-13T03:48:29.000000Z,2024-11-14T13:36:27.000000Z,in_review,2024-11-13T03:48:29.000000Z,2024-11-14T14:00:23.000000Z,34.1983,NaN,7.3989


In [13]:
proses_subject_type.shape

(861, 15)

#### Quantile Order Design

In [14]:
# Memfilter DataFrame order_process untuk mendapatkan hanya baris yang memiliki subject_type 'App\Models\OrderDesign'
quan_design = proses_subject_type[proses_subject_type['subject_type'] == 'App\Models\OrderDesign']

In [15]:
# Memilih kolom tertentu dari DataFrame quan_design untuk membuat subset baru
quan_design = quan_design[['subject_type','old_status','attributes_status','time_interval_status']]
quan_design

,subject_type,old_status,attributes_status,time_interval_status
0,App\Models\OrderDesign,None,paid,NaN
1,App\Models\OrderDesign,None,paid,NaN
20,App\Models\OrderDesign,in_review,finish,11.8122
24,App\Models\OrderDesign,paid,wait_freelancer,29.5636
32,App\Models\OrderDesign,on_going,in_review_internal,47.7178
...,...,...,...,...
1533,App\Models\OrderDesign,in_review,in_review_internal,549.6994
1536,App\Models\OrderDesign,in_review_internal,in_review,64.0092
1538,App\Models\OrderDesign,on_going,in_review_internal,78.1986
1539,App\Models\OrderDesign,new,paid,7.0289


In [16]:
# Memfilter DataFrame quan_design untuk menyertakan hanya baris yang memiliki attributes_status dan old_status dalam daftar status yang ditentukan
quan_design = quan_design[(quan_design['attributes_status'].isin(['new', 'paid', 'wait_freelancer', 'on_going', 'in_review_internal', 'in_review', 'finish', 'expired'])) & (quan_design['old_status'].isin(['new', 'paid', 'wait_freelancer', 'on_going', 'in_review_internal', 'in_review', 'finish', 'expired']))]
quan_design

,subject_type,old_status,attributes_status,time_interval_status
20,App\Models\OrderDesign,in_review,finish,11.8122
24,App\Models\OrderDesign,paid,wait_freelancer,29.5636
32,App\Models\OrderDesign,on_going,in_review_internal,47.7178
46,App\Models\OrderDesign,paid,wait_freelancer,12.0692
47,App\Models\OrderDesign,in_review_internal,in_review,15.9558
...,...,...,...,...
1532,App\Models\OrderDesign,in_review,in_review_internal,549.6994
1533,App\Models\OrderDesign,in_review,in_review_internal,549.6994
1536,App\Models\OrderDesign,in_review_internal,in_review,64.0092
1538,App\Models\OrderDesign,on_going,in_review_internal,78.1986


In [17]:
# Menggabungkan kolom old_status dan attributes_status menjadi kolom baru order_design_status dengan format 'old_status - attributes_status'
quan_design['order_design_status'] = quan_design['old_status'].astype(str) + ' - ' + quan_design['attributes_status'].astype(str)

# Menentukan posisi kolom baru (misalnya di posisi ke-1)
quan_design.insert(0, 'order_design_status', quan_design.pop('order_design_status'))
quan_design

,order_design_status,subject_type,old_status,attributes_status,time_interval_status
20,in_review - finish,App\Models\OrderDesign,in_review,finish,11.8122
24,paid - wait_freelancer,App\Models\OrderDesign,paid,wait_freelancer,29.5636
32,on_going - in_review_internal,App\Models\OrderDesign,on_going,in_review_internal,47.7178
46,paid - wait_freelancer,App\Models\OrderDesign,paid,wait_freelancer,12.0692
47,in_review_internal - in_review,App\Models\OrderDesign,in_review_internal,in_review,15.9558
...,...,...,...,...,...
1532,in_review - in_review_internal,App\Models\OrderDesign,in_review,in_review_internal,549.6994
1533,in_review - in_review_internal,App\Models\OrderDesign,in_review,in_review_internal,549.6994
1536,in_review_internal - in_review,App\Models\OrderDesign,in_review_internal,in_review,64.0092
1538,on_going - in_review_internal,App\Models\OrderDesign,on_going,in_review_internal,78.1986


In [18]:
quan_design.drop(columns=['old_status','attributes_status'],inplace=True)

In [19]:
# Memfilter DataFrame quan_design untuk menyertakan hanya baris yang memiliki order_design_status dalam daftar status yang ditentukan
quan_design = quan_design[quan_design['order_design_status'].isin(['new - paid','new - expired','paid - wait_freelancer','wait_freelancer - on_going',
                             'on_going - in_review_internal','in_review_internal - in_review','in_review - on_going','in_review - finish'])]
quan_design.drop(columns='subject_type',inplace=True)
quan_design

,order_design_status,time_interval_status
20,in_review - finish,11.8122
24,paid - wait_freelancer,29.5636
32,on_going - in_review_internal,47.7178
46,paid - wait_freelancer,12.0692
47,in_review_internal - in_review,15.9558
...,...,...
1500,on_going - in_review_internal,74.9253
1511,paid - wait_freelancer,28.2928
1536,in_review_internal - in_review,64.0092
1538,on_going - in_review_internal,78.1986


In [20]:
# Menghapus duplikat dari DataFrame quan_design, menyimpan hanya kemunculan pertama dari setiap baris yang duplikat
quan_design = quan_design.drop_duplicates(keep='first')

In [21]:

# Menghitung kuartil pertama (25th percentile), kuartil kedua (median atau 50th percentile), dan kuartil ketiga (75th percentile)
q1 = quan_design['time_interval_status'].quantile(0.25)
q2 = quan_design['time_interval_status'].quantile(0.50)
q3 = quan_design['time_interval_status'].quantile(0.75)

# Fungsi untuk memberikan label kuartil
def label_quartile(row):
    if row['time_interval_status'] <= q1:
        return 'Q1'
    elif row['time_interval_status'] <= q2:
        return 'Q2'
    elif row['time_interval_status'] <= q3:
        return 'Q3'
    else:
        return 'Q4'

# Menerapkan fungsi ke DataFrame untuk membuat kolom baru
quan_design['quartile'] = quan_design.apply(label_quartile, axis=1)


In [22]:
quan_design['count'] = quan_design.groupby('order_design_status')['quartile'].transform('count')

In [23]:
quan_design.head(10)

,order_design_status,time_interval_status,quartile,count
20,in_review - finish,11.8122,Q2,21
24,paid - wait_freelancer,29.5636,Q3,36
32,on_going - in_review_internal,47.7178,Q3,34
46,paid - wait_freelancer,12.0692,Q2,36
47,in_review_internal - in_review,15.9558,Q2,39
49,on_going - in_review_internal,7.2758,Q1,34
51,wait_freelancer - on_going,23.4908,Q3,33
117,in_review - finish,7.0089,Q1,21
119,in_review_internal - in_review,195.0047,Q4,39
123,on_going - in_review_internal,27.5767,Q3,34


#### Quantile Order Garden

In [24]:
quan_garden = proses_subject_type[proses_subject_type['subject_type'] == 'App\Models\OrderGarden']

In [25]:
quan_garden.head()

,id,subject_type,subject_id,properties,created_at,updated_at,old_status,old_created_at,old_updated_at,attributes_status,attributes_created_at,attributes_updated_at,order_cycle_time,id_datetime,time_interval_status
59,16051,App\Models\OrderGarden,210,"{""attributes"":{""id_order"":210,""id_garden"":null...",2024-11-14 09:23:20,2024-11-14 09:23:20,paid,2024-11-12T02:38:38.000000Z,2024-11-12T02:39:52.000000Z,wait_gardener,2024-11-12T02:38:38.000000Z,2024-11-14T02:23:20.000000Z,47.7450,129.0,54.7244
70,16027,App\Models\OrderGarden,211,"{""attributes"":{""id_order"":211,""id_garden"":null...",2024-11-14 09:14:19,2024-11-14 09:14:19,paid,2024-11-14T00:00:12.000000Z,2024-11-14T00:00:12.000000Z,wait_gardener,2024-11-14T00:00:12.000000Z,2024-11-14T02:14:19.000000Z,2.2353,134.0,9.2353
115,15918,App\Models\OrderGarden,211,"{""attributes"":{""id_order"":211,""id_garden"":null...",2024-11-14 07:00:12,2024-11-14 07:00:12,None,None,None,paid,2024-11-14T00:00:12.000000Z,2024-11-14T00:00:12.000000Z,0.0000,134.0,NaN
284,14354,App\Models\OrderGarden,210,"{""attributes"":{""id_order"":210,""id_garden"":null...",2024-11-12 09:39:52,2024-11-12 09:39:52,new,2024-11-12T02:38:38.000000Z,2024-11-12T02:38:38.000000Z,paid,2024-11-12T02:38:38.000000Z,2024-11-12T02:39:52.000000Z,0.0206,129.0,7.0206
285,14350,App\Models\OrderGarden,210,"{""attributes"":{""id_order"":210,""id_garden"":null...",2024-11-12 09:38:38,2024-11-12 09:38:38,None,None,None,new,2024-11-12T02:38:38.000000Z,2024-11-12T02:38:38.000000Z,0.0000,129.0,NaN


In [26]:
quan_garden = quan_garden[['subject_type','old_status','attributes_status','time_interval_status']]
quan_garden

,subject_type,old_status,attributes_status,time_interval_status
59,App\Models\OrderGarden,paid,wait_gardener,54.7244
70,App\Models\OrderGarden,paid,wait_gardener,9.2353
115,App\Models\OrderGarden,None,paid,NaN
284,App\Models\OrderGarden,new,paid,7.0206
285,App\Models\OrderGarden,None,new,NaN
...,...,...,...,...
1375,App\Models\OrderGarden,None,paid,NaN
1433,App\Models\OrderGarden,new,paid,7.0267
1434,App\Models\OrderGarden,None,new,NaN
1530,App\Models\OrderGarden,wait_gardener,acc_gardener,7.0036


In [27]:
quan_garden = quan_garden[(quan_garden['attributes_status'].isin(['new', 'paid', 'wait_gardener', 'acc_gardener', 'build_start', 'in_review', 'finish', 'expired'])) & (quan_garden['old_status'].isin(['new', 'paid', 'wait_gardener', 'acc_gardener', 'build_start', 'in_review', 'finish', 'expired']))]
quan_garden

,subject_type,old_status,attributes_status,time_interval_status
59,App\Models\OrderGarden,paid,wait_gardener,54.7244
70,App\Models\OrderGarden,paid,wait_gardener,9.2353
284,App\Models\OrderGarden,new,paid,7.0206
325,App\Models\OrderGarden,paid,wait_gardener,69.5564
344,App\Models\OrderGarden,in_review,finish,7.0028
...,...,...,...,...
1365,App\Models\OrderGarden,acc_gardener,build_start,367.9483
1368,App\Models\OrderGarden,build_start,in_review,75.5328
1433,App\Models\OrderGarden,new,paid,7.0267
1530,App\Models\OrderGarden,wait_gardener,acc_gardener,7.0036


In [28]:
quan_garden['order_garden_status'] = quan_garden['old_status'].astype(str) + ' - ' + quan_garden['attributes_status'].astype(str)

# Menentukan posisi kolom baru (misalnya di posisi ke-1)
quan_garden.insert(0, 'order_garden_status', quan_garden.pop('order_garden_status'))
quan_garden

,order_garden_status,subject_type,old_status,attributes_status,time_interval_status
59,paid - wait_gardener,App\Models\OrderGarden,paid,wait_gardener,54.7244
70,paid - wait_gardener,App\Models\OrderGarden,paid,wait_gardener,9.2353
284,new - paid,App\Models\OrderGarden,new,paid,7.0206
325,paid - wait_gardener,App\Models\OrderGarden,paid,wait_gardener,69.5564
344,in_review - finish,App\Models\OrderGarden,in_review,finish,7.0028
...,...,...,...,...,...
1365,acc_gardener - build_start,App\Models\OrderGarden,acc_gardener,build_start,367.9483
1368,build_start - in_review,App\Models\OrderGarden,build_start,in_review,75.5328
1433,new - paid,App\Models\OrderGarden,new,paid,7.0267
1530,wait_gardener - acc_gardener,App\Models\OrderGarden,wait_gardener,acc_gardener,7.0036


In [29]:
quan_garden.drop(columns=['old_status','attributes_status'],inplace=True)

In [30]:
quan_garden = quan_garden[quan_garden['order_garden_status'].isin(['new - paid','new - expired','paid - wait_gardener','wait_gardener - acc_gardener',
                             'acc_gardener - build_start','build_start - in_review','in_review - finish'])]
quan_garden.drop(columns='subject_type',inplace=True)
quan_garden

,order_garden_status,time_interval_status
59,paid - wait_gardener,54.7244
70,paid - wait_gardener,9.2353
284,new - paid,7.0206
325,paid - wait_gardener,69.5564
344,in_review - finish,7.0028
...,...,...
1365,acc_gardener - build_start,367.9483
1368,build_start - in_review,75.5328
1433,new - paid,7.0267
1530,wait_gardener - acc_gardener,7.0036


In [31]:
quan_garden = quan_garden.drop_duplicates(keep='first')

In [32]:

q1 = quan_garden['time_interval_status'].quantile(0.25)
q2 = quan_garden['time_interval_status'].quantile(0.50)
q3 = quan_garden['time_interval_status'].quantile(0.75)

# Fungsi untuk memberikan label kuartil
def label_quartile(row):
    if row['time_interval_status'] <= q1:
        return 'Q1'
    elif row['time_interval_status'] <= q2:
        return 'Q2'
    elif row['time_interval_status'] <= q3:
        return 'Q3'
    else:
        return 'Q4'

# Menerapkan fungsi ke DataFrame untuk membuat kolom baru
quan_garden['quartile'] = quan_garden.apply(label_quartile, axis=1)

In [33]:
quan_garden['count'] = quan_garden.groupby('order_garden_status')['quartile'].transform('count')

In [34]:
quan_garden.head(10)

,order_garden_status,time_interval_status,quartile,count
59,paid - wait_gardener,54.7244,Q3,20
70,paid - wait_gardener,9.2353,Q2,20
284,new - paid,7.0206,Q1,4
325,paid - wait_gardener,69.5564,Q3,20
344,in_review - finish,7.0028,Q1,13
348,acc_gardener - build_start,128.6447,Q4,15
349,build_start - in_review,62.8919,Q3,12
350,in_review - finish,7.0017,Q1,13
356,acc_gardener - build_start,178.4872,Q4,15
358,build_start - in_review,14.9572,Q2,12


#### Quantile Order Garden Care Detail

In [35]:
quan_care = proses_subject_type[proses_subject_type['subject_type'] == 'App\Models\OrderGardenCareDetail']

In [36]:
quan_care.head()

,id,subject_type,subject_id,properties,created_at,updated_at,old_status,old_created_at,old_updated_at,attributes_status,attributes_created_at,attributes_updated_at,order_cycle_time,id_datetime,time_interval_status
2,16509,App\Models\OrderGardenCareDetail,1160,"{""attributes"":{""id"":1160,""id_order_garden_care...",2024-11-14 22:36:51,2024-11-14 22:36:51,paid,2024-10-06T10:18:37.000000Z,2024-11-02T03:55:18.000000Z,acc_gardener,2024-10-06T10:18:37.000000Z,2024-11-14T15:36:51.000000Z,941.3039,NaN,306.6925
3,16478,App\Models\OrderGardenCareDetail,976,"{""attributes"":{""id"":976,""id_order_garden_care""...",2024-11-14 22:24:25,2024-11-14 22:24:25,paid,2024-09-21T05:12:37.000000Z,2024-10-30T06:31:36.000000Z,acc_gardener,2024-09-21T05:12:37.000000Z,2024-11-14T15:24:25.000000Z,1306.1967,NaN,375.8803
5,16474,App\Models\OrderGardenCareDetail,1480,"{""attributes"":{""id"":1480,""id_order_garden_care...",2024-11-14 21:00:23,2024-11-14 21:00:23,acc_gardener,2024-11-13T03:48:29.000000Z,2024-11-14T13:36:27.000000Z,in_review,2024-11-13T03:48:29.000000Z,2024-11-14T14:00:23.000000Z,34.1983,NaN,7.3989
6,16474,App\Models\OrderGardenCareDetail,1480,"{""attributes"":{""id"":1480,""id_order_garden_care...",2024-11-14 21:00:23,NaT,acc_gardener,2024-11-13T03:48:29.000000Z,2024-11-14T13:36:27.000000Z,build_start,None,None,NaN,NaN,7.3989
7,16474,App\Models\OrderGardenCareDetail,1480,"{""attributes"":{""id"":1480,""id_order_garden_care...",NaT,NaT,build_start,None,None,in_review,2024-11-13T03:48:29.000000Z,2024-11-14T14:00:23.000000Z,34.1983,NaN,NaN


In [37]:
quan_care = quan_care[['old_status','attributes_status','time_interval_status']]
quan_care

,old_status,attributes_status,time_interval_status
2,paid,acc_gardener,306.6925
3,paid,acc_gardener,375.8803
5,acc_gardener,in_review,7.3989
6,acc_gardener,build_start,7.3989
7,build_start,in_review,NaN
...,...,...,...
1523,None,new,NaN
1524,None,new,NaN
1525,None,new,NaN
1526,None,new,NaN


In [38]:
quan_care = quan_care[(quan_care['attributes_status'].isin(['new', 'paid', 'acc_gardener', 'wait_gardener','in_review'])) & (quan_care['old_status'].isin(['new', 'paid', 'acc_gardener', 'wait_gardener','in_review']))]
quan_care

,old_status,attributes_status,time_interval_status
2,paid,acc_gardener,306.6925
3,paid,acc_gardener,375.8803
5,acc_gardener,in_review,7.3989
8,paid,acc_gardener,7.0644
11,acc_gardener,paid,497.9508
...,...,...,...
1503,acc_gardener,in_review,238.6675
1506,paid,acc_gardener,7.0342
1508,paid,acc_gardener,34.5633
1510,paid,acc_gardener,772.2128


In [39]:
quan_care['order_care_status'] = quan_care['old_status'].astype(str) + ' - ' + quan_care['attributes_status'].astype(str)

# Menentukan posisi kolom baru (misalnya di posisi ke-1)
quan_care.insert(0, 'order_care_status', quan_care.pop('order_care_status'))
quan_care

,order_care_status,old_status,attributes_status,time_interval_status
2,paid - acc_gardener,paid,acc_gardener,306.6925
3,paid - acc_gardener,paid,acc_gardener,375.8803
5,acc_gardener - in_review,acc_gardener,in_review,7.3989
8,paid - acc_gardener,paid,acc_gardener,7.0644
11,acc_gardener - paid,acc_gardener,paid,497.9508
...,...,...,...,...
1503,acc_gardener - in_review,acc_gardener,in_review,238.6675
1506,paid - acc_gardener,paid,acc_gardener,7.0342
1508,paid - acc_gardener,paid,acc_gardener,34.5633
1510,paid - acc_gardener,paid,acc_gardener,772.2128


In [40]:
quan_care.drop(columns=['old_status','attributes_status'],inplace=True)

In [41]:
quan_care = quan_care[quan_care['order_care_status'].isin(['new - paid', 'paid - acc_gardener', 'acc_gardener - in_review', 'wait_gardener - in_review'])]
# quan_care.drop(columns='subject_type',inplace=True)
quan_care

,order_care_status,time_interval_status
2,paid - acc_gardener,306.6925
3,paid - acc_gardener,375.8803
5,acc_gardener - in_review,7.3989
8,paid - acc_gardener,7.0644
14,paid - acc_gardener,39.9619
...,...,...
1503,acc_gardener - in_review,238.6675
1506,paid - acc_gardener,7.0342
1508,paid - acc_gardener,34.5633
1510,paid - acc_gardener,772.2128


In [42]:
quan_care = quan_care.drop_duplicates(keep='first')

In [43]:

q1 = quan_care['time_interval_status'].quantile(0.25)
q2 = quan_care['time_interval_status'].quantile(0.50)
q3 = quan_care['time_interval_status'].quantile(0.75)

# Fungsi untuk memberikan label kuartil
def label_quartile(row):
    if row['time_interval_status'] <= q1:
        return 'Q1'
    elif row['time_interval_status'] <= q2:
        return 'Q2'
    elif row['time_interval_status'] <= q3:
        return 'Q3'
    else:
        return 'Q4'

# Menerapkan fungsi ke DataFrame untuk membuat kolom baru
quan_care['quartile'] = quan_care.apply(label_quartile, axis=1)

In [44]:
quan_care['count'] = quan_care.groupby('order_care_status')['quartile'].transform('count')

In [45]:
quan_care.head(10)

,order_care_status,time_interval_status,quartile,count
2,paid - acc_gardener,306.6925,Q3,115
3,paid - acc_gardener,375.8803,Q4,115
5,acc_gardener - in_review,7.3989,Q1,41
8,paid - acc_gardener,7.0644,Q1,115
14,paid - acc_gardener,39.9619,Q2,115
15,paid - acc_gardener,18.3208,Q2,115
18,paid - acc_gardener,18.1803,Q2,115
25,acc_gardener - in_review,62.3719,Q3,41
29,acc_gardener - in_review,57.1778,Q3,41
52,paid - acc_gardener,1800.4833,Q4,115
